In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
def read_csv(file):
	return pd.read_csv(file)

def drop_duplicates(df):
	return df.drop_duplicates(subset=None, keep='first', inplace=False)

def sort_by_date(df):
	return df.sort_values(by=['Sale Date'], inplace=False, ascending=True)

def process_datetime(df):
	df['Sale Date'] = df['Sale Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
	return df

def get_year_month_week(df):
	df['Year'] = df['Sale Date'].apply(lambda x: x.year)
	df['Month'] = df['Sale Date'].apply(lambda x: x.month)
	df['Week'] = df['Sale Date'].apply(lambda x: x.isocalendar()[1])
	return df

def groupd_df_by_year_and_month(df):
	return df.groupby(['Year','Month'])

def get_unique_value_counts(df):
	return df.nunique()

def separate_sales_and_return(df):
	return df[df['Sale/Return'] == 'Sale'], df[df['Sale/Return'] == 'Return']

In [3]:
def check_sales_quantity_anomalies(df):
    df = df[df['Sales Qty'] != 0]
    df = df[(df['Sale/Return']=='Sale') & (df['Sales Qty'] > 0)]
    df = df[(df['Sale/Return']=='Return') & (df['Sales Qty'] < 0)]
    return df

def check_mrp_sales_price_find_anomalies(df):
	no_anomalies_df = df[df['MRP'] == df['Sales Price']]
	discounted_df = df[df['MRP'] > df['Sales Price']]
	need_correction_df = df[df['MRP'] < df['Sales Price']]
	return no_anomalies_df, discounted_df, need_correction_df

def correct_mrp_sales_price_anomalies(need_correction_df):
	need_correction_df['Sales Price'] = need_correction_df['MRP']
	return need_correction_df

In [4]:
def calculate_discounts(discounted_df):
	discounted_df['Discount'] = (discounted_df['MRP'] - discounted_df['Sales Price'])*discounted_df['Sales Qty']
	return discounted_df

def calculate_sales(df):
	df['Sales'] = df['Sales Price']*df['Sales Qty']
	return df

def get_sales_metrics_by_year_and_month(df):
	return df.groupby(['Year','Month'])['Sales'].sum()

def get_discount_metrics_by_year_and_month(df):
    sales_df = get_sales_metrics_by_year_and_month(df)
    discounts_df = df.groupby(['Year','Month'])['Discount'].sum()
    return discounts_df/sales_df.values * 100    

def get_average_selling_price_by_year_and_month(df):
    return df.groupby(['Year','Month'])['Sales Price'].mean()

def get_unique_SKU_by_year_and_month(df):
    return df.groupby(['Year','Month'])['SKU Code'].nunique()

def get_sales_metrics_by_category_year_and_month(df):
	return df.groupby(['Category','Brand Code','Year','Month'])['Sales'].sum().reset_index()

def get_discount_metrics_by_category_year_and_month(df):
	return df.groupby(['Category','Brand Code','Year','Month'])['Discount'].sum().reset_index()

def get_average_selling_price_by_category_year_and_month(df):
    return df.groupby(['Category','Brand Code','Year','Month'])['Sales Price'].mean().reset_index()

def get_unique_SKU_by_category_year_and_month(df):
    return df.groupby(['Category','Brand Code','Year','Month'])['SKU Code'].nunique().reset_index()

In [5]:
def get_files():
	this_dir = os.getcwd()
	parent_dir = os.path.dirname(os.path.normpath(this_dir))
	data_dir = os.path.join(parent_dir, 'data')
	files = [os.path.join(data_dir, x) for x in os.listdir(data_dir)]
	files.sort()
	return files

def merge_stores_data(files):
	for i,file in enumerate(files):
		if i == 0:
			df = read_csv(file)
		else:
			df = df.append(read_csv(file))
	return df

def split_stores_into_dfs(df):
	stores = np.unique(df['Store Code'].values)
	stores.sort()
	store_dfs = {}
	for store in stores:
		store_dfs[store] = df[df['Store Code'] == store]
	return store_dfs

def preprocess(df):
    df = process_datetime(df)
    df = get_year_month_week(df)
    df = check_sales_quantity_anomalies(df)
    no_anomalies_df, discounted_df, need_correction_df = check_mrp_sales_price_find_anomalies(df)
    corrected_df = correct_mrp_sales_price_anomalies(need_correction_df)
    no_anomalies_df['Discount'] = 0.
    corrected_df['Discount'] = 0.
    discounted_df = calculate_discounts(discounted_df)
    preprocessed_df = no_anomalies_df.append(corrected_df)
    preprocessed_df = preprocessed_df.append(discounted_df)
    preprocessed_df = calculate_sales(preprocessed_df)
    preprocessed_df = sort_by_date(preprocessed_df)
    return preprocessed_df

def get_preprocessed_store_dfs():
    files = get_files()
    df = merge_stores_data(files)
    preprocessed_df = preprocess(df)
    store_dfs = split_stores_into_dfs(preprocessed_df)
    return store_dfs

In [6]:
def get_sales_discount_selling_price_metrics_by_store_year_month(store_dfs):
    sales_metrics = {}
    discount_metrics = {}
    selling_price_metrics = {}
    sku_metrics = {}
    for store in store_dfs:
        sales_metrics[store] = get_sales_metrics_by_year_and_month(store_dfs[store])
        discount_metrics[store] = get_discount_metrics_by_year_and_month(store_dfs[store])
        selling_price_metrics[store] = get_average_selling_price_by_year_and_month(store_dfs[store])
        sku_metrics[store] = get_unique_SKU_by_year_and_month(store_dfs[store])
    return sales_metrics, discount_metrics, selling_price_metrics, sku_metrics

def get_sales_discount_selling_price_metrics_by_store_category_year_month(store_dfs):
    sales_metrics = {}
    discount_metrics = {}
    selling_price_metrics = {}
    sku_metrics = {}
    for store in store_dfs:
        sales_metrics[store] = get_sales_metrics_by_category_year_and_month(store_dfs[store])
        discount_metrics[store] = get_discount_metrics_by_category_year_and_month(store_dfs[store])
        selling_price_metrics[store] = get_average_selling_price_by_category_year_and_month(store_dfs[store])
        sku_metrics[store] = get_unique_SKU_by_category_year_and_month(store_dfs[store])
    return sales_metrics, discount_metrics, selling_price_metrics, sku_metrics

In [7]:
store_dfs = get_preprocessed_store_dfs()
sales_metrics, discount_metrics, selling_price_metrics, sku_metrics = get_sales_discount_selling_price_metrics_by_store_year_month(store_dfs)
deep_dive_sales_metrics, deep_dive_discount_metrics, deep_dive_selling_price, deep_dive_sku_metrics = get_sales_discount_selling_price_metrics_by_store_category_year_month(store_dfs)

In [8]:
this_dir = os.getcwd()
parent_dir = os.path.dirname(os.path.normpath(this_dir))
preproc_dir = os.path.join(parent_dir, 'preproc_files')
if not os.path.exists(preproc_dir):
    os.makedirs(preproc_dir)
f_store_dfs = os.path.join(preproc_dir, 'store_dfs')
f_sales_metrics = os.path.join(preproc_dir, 'sales_metrics')
f_discount_metrics = os.path.join(preproc_dir, 'disc_metrics')
f_selling_price_metrics = os.path.join(preproc_dir, 'sale_price_metrics')
f_sku_metrics = os.path.join(preproc_dir, 'sku_metrics')
f_sales_metrics_deep = os.path.join(preproc_dir, 'sales_metrics_deep')
f_discount_metrics_deep = os.path.join(preproc_dir, 'disc_metrics_deep')
f_selling_price_metrics_deep = os.path.join(preproc_dir, 'sale_price_metrics_deep')
f_sku_metrics_deep = os.path.join(preproc_dir, 'sku_metrics_deep')
files = [f_store_dfs,
         f_sales_metrics,
         f_discount_metrics,
         f_selling_price_metrics,
         f_sku_metrics,
         f_sales_metrics_deep,
         f_discount_metrics_deep,
         f_selling_price_metrics_deep,
         f_sku_metrics_deep]
data = [store_dfs,
        sales_metrics,
        discount_metrics,
        selling_price_metrics,
        sku_metrics,
        deep_dive_sales_metrics,
        deep_dive_discount_metrics,
        deep_dive_selling_price,
        deep_dive_sku_metrics]
for i,file in enumerate(files):
    with open(file, 'wb') as f:
        pickle.dump(data[i], f)